### Import

In [1]:
import numpy as np
import pandas as pd

### Fetch the data from the Internet and save it as csv
The following cell only needs to run once. 

In [2]:
# from ucimlrepo import fetch_ucirepo 

# # fetch dataset 
# breast_cancer_wisconsin_original = fetch_ucirepo(id=15) 
  
# # data (as pandas dataframes) 
# X = breast_cancer_wisconsin_original.data.features 
# y = breast_cancer_wisconsin_original.data.targets 

# data = pd.concat([y, X], axis=1)
# data.to_csv("data.csv", index=False)

### Read the data to X and y, and replace B and M (2&4) with 0 and 1

Note that there are missing values. As we got a large dataset, we will just drop the records.

In [2]:
data = pd.read_csv("data.csv")
data.dropna(inplace=True)
X = data.drop("Class", axis=1)
y = data['Class']
y.replace({2: 0, 4: 1}, inplace=True)
print(X.shape, y.shape)

(683, 9) (683,)


In [290]:
a = np.array([1,2,3]).reshape(-1,1)
b=np.array([4,5,6]).reshape(1, -1)
print(a.shape)
print(b.shape)
print(a.dot(b).shape)

(3, 1)
(1, 3)
(3, 3)


In [320]:
from numpy import random
import matplotlib.pyplot as plt


class Layer:
    num_of_layers = 0
    def __init__(self, input_size, output_size, activation="linear"):
        self.weights = random.rand(input_size+1, output_size)-0.5   # weight[0, x] is bias's weight
        # self.weights[:,:] = 1
        self.input_size = input_size
        self.output_size = output_size
        self.activation = activation
        if activation == "linear":
            self.activation_function = lambda x: x
            self.d_activation_function = lambda x: 1
        elif activation == "sigmoid":
            self.activation_function = lambda x: 1/(1+np.exp(-x))
            # self.d_activation_function = lambda x: 
        else:
            raise Exception("Wrong activation function")
        Layer.num_of_layers += 1

    def forward_propagation(self, input):   # input should be an array
        input = np.concatenate(([[1], input]))    # add bias = 1
        self.input = input
        output = self.activation_function(input.dot(self.weights))
        self.output = output
        return output
    
    def backward_propagation(self, dl_dy, learning_rate = 0.0002):
        if self.activation == "linear":
            d_da = 1
        elif self.activation == "sigmoid":
            pass
        # print(f"dl_dy.shape: {dl_dy.shape}")
        # print(f"self.weights.shape: {self.weights.shape}")

        da_dw = self.input.reshape(-1, 1)
        # print(f"da_dw.shape: {da_dw.shape}")

        dl_dw = da_dw.dot(dl_dy.T * d_da)
        # print(f"dl_dw.shape: {dl_dw.shape}")

        dl_dx = (self.weights[1:]).dot(dl_dy * d_da)
        # print(f"dl_dx.shape: {dl_dx.shape}")

        self.weights -= learning_rate * dl_dw

        # print()
        return dl_dx
        

class Model:
    def __init__(self):
        self.layers = []

    def add(self, layer):
        self.layers.append(layer)

    def show_layers(self):
        for i, l in enumerate(self.layers):
            print(f"Layer {i+1}: ", end="")
            print(f"input size: {l.input_size}, output size: {l.output_size}")

    def fp1(self, input):
        prev_output = input
        for layer in self.layers:
            prev_output = layer.forward_propagation(prev_output)
            print(prev_output)
        return prev_output

    def one_cycle(self, input, expected_output):
        pred = self.fp1(input)
        error = (expected_output - pred)
        sq_error = (expected_output - pred)**2
        derror_dpred = -2 * error
        derror_dpred = np.array(derror_dpred).reshape(1, 1)
        dl_dy = derror_dpred
        for layer in reversed(self.layers):
            dl_dy = layer.backward_propagation(dl_dy)

        print()
    
layer1 = Layer(5, 3, activation="linear")
layer2 = Layer(3, 2, activation="linear")
layer3 = Layer(2, 1, activation="linear")


model = Model()
model.add(layer1)
model.add(layer2)
model.add(layer3)
model.show_layers()

input = np.array([5, 8, 3, 2, 1])
expected_output = sum(input*[3, 4, 6, 2, 1])+5
print(expected_output)


for i in range(20):
    model.one_cycle(input, expected_output)

Layer 1: input size: 5, output size: 3
Layer 2: input size: 3, output size: 2
Layer 3: input size: 2, output size: 1
75
[-5.51171724  3.7885127   3.51284415]
[1.8843869  3.58148066]
[-0.84304831]

[-5.20376347  3.76446961  3.15433602]
[1.68804385 2.91174327]
[-0.23492703]

[-5.0784975   3.75476497  2.99209297]
[1.65542194 2.60864505]
[0.14507756]

[-5.08024055  3.74101878  2.97459164]
[1.74572682 2.54761618]
[0.46832078]

[-5.1925814   3.71979515  3.08439853]
[1.9554449 2.6915306]
[0.83364981]

[-5.42229981  3.69685619  3.32363778]
[2.30669991 3.05136438]
[1.34045535]

[-5.79698575  3.68667933  3.71129963]
[2.85367578 3.68534452]
[2.14143637]

[-6.36998643  3.71501747  4.28656277]
[3.70593307 4.72386686]
[3.53112536]

[-7.23208118  3.82439827  5.11738746]
[5.08141931 6.43260795]
[6.15359783]

[-8.52810884  4.08215927  6.31268102]
[7.41625582 9.34576746]
[11.53488423]

[-10.45354777   4.58226916   8.01726958]
[11.53419342 14.47223834]
[23.33581058]

[-13.06908911   5.38106613  10.257664